In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import datetime
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
#from lightgbm import LGBMRegressor

#from gensim.models import Word2Vec
#from gensim.models import KeyedVectors
#from sklearn.manifold import TSNE
#from sklearn.decomposition import PCA

warnings.filterwarnings(action='ignore')

In [3]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
submission = pd.read_csv('data/sample_submission.csv')

In [4]:
# Basic text preprocessing

def split_process(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        elif '*' in i:
            x_.extend(i.split('*'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    return x_

In [11]:
food_combinations = []
for i in ['조식메뉴', '중식메뉴', '석식메뉴']:
    #food_combinations += train[i].apply(lambda x: split_process(x)).to_list()
    food_combinations += test[i].apply(lambda x: split_process(x)).to_list()

In [10]:
train['조식메뉴2'] = train[i].apply(lambda x: split_process(x, i))

In [16]:
food_combinations

[['계란후라이',
  '단호박죽',
  '고기완자전',
  '사과',
  '연유버터베이글',
  '흑미밥',
  '양상추샐러드',
  '우거지국',
  '우유',
  '포기김치',
  '주스',
  '모닝롤',
  '찐계란'],
 ['계란후라이',
  '찐계란',
  '황태국',
  '누룽지탕',
  '흑미밥',
  '시래기지짐',
  '양상추샐러드',
  '우유',
  '포기김치',
  '주스',
  '모닝롤',
  '대만샌드위치'],
 ['고구마순볶음',
  '계란후라이',
  '흑미밥',
  '모닝롤',
  '오곡죽',
  '양상추샐러드',
  '우유',
  '포기김치',
  '주스',
  '핫케익',
  '매생이굴국',
  '찐계란'],
 ['계란후라이',
  '누룽지탕',
  '흑미밥',
  '두부김칫국',
  '양상추샐러드',
  '우유',
  '새우완자전',
  '포기김치',
  '주스',
  '모닝롤',
  '찐계란',
  '촉촉한치즈케익'],
 ['계란후라이',
  '호박맑은국',
  '채소죽',
  '흑미밥',
  '토마토샌드',
  '양상추샐러드',
  '우유',
  '포기김치',
  '주스',
  '모닝롤',
  '오이생채',
  '찐계란'],
 ['계란후라이',
  '흑미밥',
  '바나나',
  '베이컨숙주볶음',
  '양상추샐러드',
  '시래기된장국',
  '우유',
  '소고기죽',
  '포기김치',
  '주스',
  '모닝롤',
  '찐계란',
  '게살모닝샌드'],
 ['계란후라이',
  '누룽지탕',
  '흑미밥',
  '양상추샐러드',
  '사과파이',
  '우유',
  '아욱국',
  '새송이버섯곤약장조림',
  '주스',
  '모닝롤',
  '포기김치',
  '찐계란'],
 ['고구마죽',
  '계란후라이',
  '머위나물무침',
  '흑미밥',
  '옹심이국',
  '양상추샐러드',
  '우유',
  '포기김치',
  '주스',
  '모닝롤',
  '찐계란',
  '앙버터모닝빵'],
 ['계란후라이',
  '누룽지탕

In [17]:
food_all = [element for array in food_combinations for element in array]

In [18]:
food_all2 = list(set(food_all))

In [19]:
food_df = pd.DataFrame(food_all2)
food_df

,0
0,해물우동볶음
1,솎음열무나물무침
2,매콤해물볶음
3,짬뽕수제비
4,해물탕
...,...
580,연근조림
581,수제오미산적
582,묵은지닭찜
583,오곡죽


In [20]:
food_df.to_csv("data/menu_test.csv",encoding="utf-8-sig")

In [20]:
with open("menu.txt", "w") as output:
    output.write(str(food_all2))

In [11]:
train.head()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계,조식메뉴2
0,2016-02-01,월,2601,50,150,238,0.0,모닝롤/찐빵 우유/두유/주스 계란후라이 호두죽/쌀밥 (쌀:국내산) 된장찌개 쥐...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 오징어찌개 쇠불고기 (쇠고기:호주산) 계란찜 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 육개장 자반고등어구이 두부조림 건파래무침 ...",1039.0,331.0,"[건파래무침, 포기김치, 두부조림, 쌀밥, 잡곡밥, 자반고등어구이, 육개장]"
1,2016-02-02,화,2601,50,173,319,0.0,모닝롤/단호박샌드 우유/두유/주스 계란후라이 팥죽/쌀밥 (쌀:국내산) 호박젓국찌...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 김치찌개 가자미튀김 모둠소세지구이 마늘쫑무...","콩나물밥*양념장 (쌀,현미흑미:국내산) 어묵국 유산슬 (쇠고기:호주산) 아삭고추무...",867.0,560.0,"[콩나물밥*양념장, 포기김치, 어묵국, 바나나, 유산슬, 아삭고추무침]"
2,2016-02-03,수,2601,56,180,111,0.0,모닝롤/베이글 우유/두유/주스 계란후라이 표고버섯죽/쌀밥 (쌀:국내산) 콩나물국...,"카레덮밥 (쌀,현미흑미:국내산) 팽이장국 치킨핑거 (닭고기:국내산) 쫄면야채무침 ...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 청국장찌개 황태양념구이 (황태:러시아산) 고기...",1017.0,573.0,"[포기김치, 새송이버섯볶음, 쌀밥, 잡곡밥, 청국장찌개, 황태양념구이, 고기전]"
3,2016-02-04,목,2601,104,220,355,0.0,"모닝롤/토마토샌드 우유/두유/주스 계란후라이 닭죽/쌀밥 (쌀,닭:국내산) 근대국...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 쇠고기무국 주꾸미볶음 부추전 시금치나물 ...","미니김밥*겨자장 (쌀,현미흑미:국내산) 우동 멕시칸샐러드 군고구마 무피클 포...",978.0,525.0,"[군고구마, 우동, 포기김치, 미니김밥*겨자장, 멕시칸샐러드, 무피클]"
4,2016-02-05,금,2601,278,181,34,0.0,모닝롤/와플 우유/두유/주스 계란후라이 쇠고기죽/쌀밥 (쌀:국내산) 재첩국 방...,"쌀밥/잡곡밥 (쌀,현미흑미:국내산) 떡국 돈육씨앗강정 (돼지고기:국내산) 우엉잡채...","쌀밥/잡곡밥 (쌀,현미흑미:국내산) 차돌박이찌개 (쇠고기:호주산) 닭갈비 (닭고기:...",925.0,330.0,"[포기김치, 닭갈비, 쌀밥, 잡곡밥, 감자소세지볶음, 콩나물무침, 차돌박이찌개]"


In [ ]:
# HYPER PARAMETERS

class CFG:
    emb_dim = 200

args = CFG

In [ ]:
# Train or load w2v model

TRAIN_W2V = True
try:
    model = Word2Vec.load('/content/drive/MyDrive/Colab Notebooks/구내식당 식수인원/food_embedding.model')
    print("Model loaded")
except:
    if TRAIN_W2V:
        print("Training w2v")
        model = Word2Vec(sentences=food_combinations, size=args.emb_dim, window=7, min_count=0, workers=4, sg=0)
        model.save('/content/drive/MyDrive/Colab Notebooks/구내식당 식수인원/food_embedding.model')
    else:
        print("Model loading failed. Do not train.")

In [ ]:
# w2v demo
model.wv.most_similar('된장찌개')

In [ ]:
def process_date(df):
    df['일자'] = pd.to_datetime(df['일자'], format="%Y-%m-%d")
    df['year'] = df['일자'].dt.year
    df['month'] = df['일자'].dt.month
    df['day'] = df['일자'].dt.day
    df = df.drop('일자', axis=1)
    return df

def get_food_embedding(x):
    x_ = []
    x = x.split(' ')
    for i in x:
        if '(' in i and ':' in i and ')' in i:
            continue
        if '/' in i:
            x_.extend(i.split('/'))
        else:
            x_.append(i)
    x_ = list(set(x_))
    x_.remove('')
    vec_ = np.zeros(args.emb_dim)
    for i in x_:
        vec = model.wv.get_vector(i)
        vec_ += vec
    vec_ /= len(x_)
    return vec_

In [ ]:
# General preprocessing
df_train = process_date(train)
day_encoder = LabelEncoder()
df_train['요일'] = day_encoder.fit_transform(df_train['요일'])

In [ ]:
# Get embedding
df_train['조식메뉴_embedding'] = df_train['조식메뉴'].apply(lambda x: get_food_embedding(x))
df_train['중식메뉴_embedding'] = df_train['중식메뉴'].apply(lambda x: get_food_embedding(x))
df_train['석식메뉴_embedding'] = df_train['석식메뉴'].apply(lambda x: get_food_embedding(x))

In [ ]:
y_lunch = df_train['중식계']
y_dinner = df_train['석식계']
df_train.drop(['조식메뉴', '중식메뉴', '석식메뉴', '중식계', '석식계'], axis=1, inplace=True)

In [ ]:
X_common = df_train.iloc[:, :9]

In [ ]:
emb_arr_lunch = np.array(df_train.iloc[:, 10].to_numpy().tolist()) # Ver 2
emb_arr_dinner = np.array(df_train.iloc[:, 11].to_numpy().tolist()) # Ver 2

X_train_lunch = np.concatenate((X_common.to_numpy(), emb_arr_lunch), axis=1)
X_train_dinner = np.concatenate((X_common.to_numpy(), emb_arr_dinner), axis=1)

In [ ]:
X_train_lunch, X_test_lunch, y_train_lunch, y_test_lunch = train_test_split(X_train_lunch, y_lunch, test_size=0.1, random_state=42)
X_train_dinner, X_test_dinner, y_train_dinner, y_test_dinner = train_test_split(X_train_dinner, y_dinner, test_size=0.1, random_state=42)